# Multi-Class Klassifikation

Der Unterschied zwischen einer binären und einer multi-class Klassifikation besteht in der Anzahl der Klassen, die vom Modell vorhergesagt werden sollen.

Bei einer binären Klassifikation wird das Modell trainiert, um zwischen nur zwei Klassen zu unterscheiden, z.B. Ja/Nein, 0/1 oder positiv/negativ. Ein bekanntes Beispiel hierfür ist die Spam-Erkennung bei E-Mails, bei der das Modell zwischen Spam und Nicht-Spam unterscheidet.

Im Gegensatz dazu geht es bei einer multi-class Klassifikation darum, das Modell zu trainieren, um zwischen mehr als zwei Klassen zu unterscheiden. Ein Beispiel hierfür ist die Klassifikation von verschiedenen Tierarten wie Hunden, Katzen und Vögeln. Hier muss das Modell trainiert werden, um jedes Tier korrekt zu identifizieren.

# Daten laden

In [ ]:
import pandas as pd
automaten = pd.read_csv("https://www.offenedaten-koeln.de/sites/default/files/psa_offene_daten_2023.csv", encoding='unicode_escape', sep=";")

strafen = pd.read_csv("https://www.offenedaten-koeln.de/sites/default/files/Bussgeld_2021.csv", encoding='unicode_escape', sep=";")

tatbestände = pd.read_csv("https://raw.githubusercontent.com/jomo/bkat-owi/master/data.csv")

# Datensatz vorbereiten

In [ ]:
tatbestände_aktuell = tatbestände.drop_duplicates("Tatbestandsnummer",keep="last").set_index("Tatbestandsnummer")
joined = strafen.join(tatbestände_aktuell[["Euro","Tatbestand_Druckdatei","Klassifizierung"]], on="tatbestand1")

Außerdem brauchen wir eine Zielvariable. Der einfachheithalber versuchen wir einfach nur vorherzusagen, ob es sich um Parken ohne Parkticket handelt oder nicht. Prinzipiell könnten wir aber alles mögliche versuchen vorherzusagen, solange diese Informationen verfügbar sind.

In [ ]:
joined["no_parking_ticket"] = joined.Tatbestand_Druckdatei == "Sie parkten im Bereich eines Parkscheinautomaten ohne gültigen\nParkschein."

In [ ]:
joined.no_parking_ticket.value_counts()

# Erstellen einer Zielvariable mit Hilfe von ChatGPT

In [ ]:
long_versions = joined.Tatbestand_Druckdatei.value_counts().head(50).index.tolist()

In [ ]:
# Prompt for ChatGPT
prompt = f"""Hier ist eine Liste an Vergehen eines Strafenkatalogs. Erstelle 5 Überbegriffe (maximal 3 Wörter) für die unterschiedlichen Arten an Vergehen. Ersetze anschließend in der Liste der Vergehen jedes einzelne Vergehen durch den am besten passenden Überbegriff.
 
Die Liste der Vergehen ist:   
{long_versions}"""
print(prompt)

In [ ]:
chatgpt_results = ["Parkvergehen", "Parkvergehen", "Parkvergehen", "Parkvergehen", "Parkvergehen", "Parkvergehen", "Parkvergehen", "Parkvergehen", "Sicherheitsvergehen", "Parkvergehen", "Parkvergehen", "Parkvergehen", "Parkvergehen", "Sicherheitsvergehen", "Verkehrsvergehen", "Parkvergehen", "Haltevergehen", "Parkvergehen", "Haltevergehen", "Parkvergehen", "Gehwegvergehen", "Haltevergehen", "Parkvergehen", "Gehwegvergehen", "Haltevergehen", "Gehwegvergehen", "Haltevergehen", "Parkvergehen", "Haltevergehen", "Parkvergehen", "Gehwegvergehen", "Parkvergehen", "Haltevergehen", "Gehwegvergehen", "Verkehrsvergehen", "Gehwegvergehen", "Parkvergehen", "Verkehrsvergehen", "Gehwegvergehen", "Parkvergehen", "Gehwegvergehen", "Haltevergehen", "Haltevergehen", "Gehwegvergehen", "Parkvergehen", "Gehwegvergehen", "Haltevergehen", "Parkvergehen", "Gehwegvergehen", "Parkvergehen", "Parkvergehen", "Gehwegvergehen", "Parkvergehen", "Gehwegvergehen", "Haltevergehen", "Gehwegvergehen", "Haltevergehen", "Gehwegvergehen", "Haltevergehen", "Parkvergehen", "Gehwegvergehen", "Parkvergehen", "Gehwegvergehen", "Parkvergehen", "Gehwegvergehen"]
group_mapping = dict(zip(long_versions, chatgpt_results))
joined["group"] = joined.Tatbestand_Druckdatei.replace(group_mapping)
joined.loc[~joined["group"].isin(chatgpt_results),"group"] = "Sonstiges"

# Re-Use what you already know

And Predict the "tatbestand" column

In [ ]:
joined.group.value_counts()